In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams['font.family'] = "MS Gothic"
plt.rcParams['font.size'] = 15

downsample=5

blacklist = []
blacklist.append("realdata/20211001_203702_00000001\\0000000B.TXT")


In [2]:
def pvmv_from_log(fn):
    df = pd.read_table(fn,encoding='utf-16',sep='\t',skiprows =[0,1,3])
    
    pv=[]
    mv=[]
    info=[]
    # 11	D8010	目標値	SP 送気温度（PH)	10^-2	℃	
    # 12	D8011	目標値	SP PH後温度	10^-2	℃	
    # 13	D8012	目標値	SP 送気温度（RH)	10^-2	℃	
    # 14	D8013	目標値	SP 送気絶対湿度	10^-5	kg/kg	
    # 15	D8014	目標値	SP 高圧加湿器ポンプ圧力	10^-2	Mpa	
    # 16	D8022	PID PHマスタ	PV 送気温度調節計 PH	10^-2	℃	
    # 17	D8023	PID PHマスタ	SP 送気温度調節計 PH	10^-2	℃	
    # 18	D8024	PID PHマスタ	MV 送気温度調節計 PH	10^-1	%	
    # 19	D8032	PID PHスレイブ	PV PH後温度調節計	10^-2	℃	
    # 20	D8033	PID PHスレイブ	SP PH後温度調節計	10^-2	℃	
    # 22	D8042	PID 送気温度	PV 送気温度調節計	10^-2	℃	
    # 23	D8043	PID 送気温度	SP 送気温度調節計	10^-2	℃	
    # 26	D8052	PID 送気絶対湿度	PV 絶対湿度調節計 	10^-5	kg/kg	
    # 27	D8053	PID 送気絶対湿度	SP 絶対湿度調節計 	10^-5	kg/kg	
    # 28	D8055	PID 送気絶対湿度	MV(加熱) 絶対湿度調節計 	10^-1	%	未使用
    # 32	D8062	PID高圧加湿器ポンプ圧力	PV 高圧加湿器ポンプ圧力調節計	10^-2	Mpa	
    # 33	D8063	PID高圧加湿器ポンプ圧力	SP 高圧加湿器ポンプ圧力調節計	10^-2	Mpa	
    
    pv.append(df["D8000"].values * 0.01) # 4	D8000	センサ入力	送気温度	10^-2	℃	
    pv.append(df["D8001"].values * 0.01) # 5	D8001	センサ入力	送気湿度	10^-2	％	
    pv.append(df["D8002"].values * 0.01) # 6	D8002	センサ入力	PH後温度	10^-2	℃	
    pv.append(df["D8003"].values * 0.01) # 7	D8003	センサ入力	壁温度	10^-2	℃    
    pv.append(df["D8004"].values * 0.01) # 8	D8004	センサ入力	外気温度	10^-2	℃	    
    pv.append(df["D8005"].values * 0.01) # 9	D8005	センサ入力	外気湿度	10^-2	％
    mv_mask = df["D8070.1"].values # 31	D8070.1	稼働	空調運転中	Bit		1:設備運転中   
    mv.append(mv_mask*df["D8034"].values * 0.1) # 21	D8034	PID PHスレイブ	MV PH後温度調節計	10^-1	%	PH開度指令値    
    mv.append(mv_mask*df["D8045"].values * 0.1) # 24	D8045	PID 送気温度	MV(加熱) 送気温度調節計	10^-1	%	RH開度指令値
    temp1 = df["D8046"].values * 0.1 # 25	D8046	PID 送気温度	MV(冷却) 送気温度調節計	10^-1	%	CC開度指令値（D8046>D8056）
    temp2 = df["D8056"].values * 0.1 # 29	D8056	PID 送気絶対湿度	MV(冷却) 絶対湿度調節計 	10^-1	%	CC開度指令値（D8046＜D8056）
    mv.append(mv_mask*np.stack([temp1, temp2],axis=0).max(axis=0))
    mv.append(mv_mask*df["D8080"].values) # 30	D8080	高圧加湿器	高圧加湿器電磁弁個数	10^0	個	0~８台（外気と目標の絶対湿度差からTEGAにて決定）
    
    info.append(df["D8070.1"].values)
    pv = np.array(pv).T
    mv = np.array(mv).T
    info = np.array(info).T
    return pv, mv, info


def downsampling(pv, mv, info, downsample):
    _pv = []
    _mv = []
    _info = []
    for i in range(pv.shape[0]//downsample):
        _pv.append(pv[i*downsample : (i+1)*downsample].mean(axis=0))
        _mv.append(mv[i*downsample : (i+1)*downsample].mean(axis=0))    
        _info.append(info[i*downsample : (i+1)*downsample].mean(axis=0))    
    return np.array(_pv), np.array(_mv), np.array(_info)




In [3]:
import glob
logs_list = []
logs_list += glob.glob('realdata/20210907_163440_00000001/*')
logs_list += glob.glob('realdata/20210910_185237_00000001/*')
logs_list += glob.glob('realdata/20210917_190118_00000001/*')
logs_list += glob.glob('realdata/20210925_123940_00000001/*')
logs_list += glob.glob('realdata/20211001_203702_00000001/*')

logs_list = [fn for fn in logs_list if fn not in blacklist]


pv_part_list=[]
mv_part_list=[]

for fn in tqdm(logs_list):
    
    pv, mv, info = pvmv_from_log(fn)
    pv, mv, info = downsampling(pv, mv, info, downsample)
    temp_pv_part=[]
    temp_mv_part=[]    
    for i in range(pv.shape[0]):
        if info[i]>0.9999:
            temp_pv_part.append(pv[i])
            temp_mv_part.append(mv[i])
        else:
            if len(temp_pv_part)>0:
                pv_part_list.append(np.array(temp_pv_part))
                mv_part_list.append(np.array(temp_mv_part))            
            temp_pv_part=[]
            temp_mv_part=[]
    if len(temp_pv_part)>0:
        pv_part_list.append(np.array(temp_pv_part))
        mv_part_list.append(np.array(temp_mv_part))            
        

100%|████████████████████████████████████████████████████████████████████████████████| 265/265 [03:31<00:00,  1.25it/s]


In [4]:
dir = "processeddata"+str(downsample)
if not os.path.exists(dir):
    os.makedirs(dir)

y_list=[]
u_list=[]
init_flag=1
for i in tqdm(range(len(pv_part_list))):
    y = (pv_part_list[i])[:,3:4]
    u = mv_part_list[i]
    #u = np.concatenate([mv_part_list[i], (pv_part_list[i])[:,4:]],axis=1)
    y_list.append(y)
    u_list.append(u)
    if init_flag==1:
        y_concat = y[:]
        u_concat = u[:]
        init_flag=0
    else:
        y_concat = np.concatenate([y_concat, y],axis=0)
        u_concat = np.concatenate([u_concat, u],axis=0)    
    
for i in tqdm(range(len(pv_part_list))):
    np.savetxt(dir+"\\y"+str(i+1)+".csv", y_list[i]-y_concat.mean(axis=0), delimiter=",")
    np.savetxt(dir+"\\u"+str(i+1)+".csv", u_list[i]-u_concat.mean(axis=0), delimiter=",")

100%|██████████████████████████████████████████████████████████████████████████████████| 83/83 [00:18<00:00,  4.53it/s]
